In [131]:
from flair.data import Sentence
from flair.models import SequenceTagger
from flair.splitter import SegtokSentenceSplitter
from pprint import pprint

text = """
Ein besonderer Tag im Park

Es war ein sonniger Samstagmorgen, als Anna beschloss, einen Tag im Park zu verbringen. Sie packte ihren Rucksack mit einer Decke, einem Buch und etwas zu essen. Als sie ankam, fand sie einen schönen Platz unter einem großen Baum.

Anna setzte sich und begann, ihr Buch zu lesen. Die Vögel zwitscherten fröhlich, und die Sonne schien warm auf ihr Gesicht. Nach einer Weile hörte sie Kinder lachen und sah eine Gruppe von Kindern, die Fußball spielten. Es erinnerte sie an ihre eigene Kindheit.

Plötzlich kam ein kleiner Hund auf sie zugelaufen und begann, an ihrer Decke zu schnüffeln. Anna lachte und streichelte den Hund. Bald darauf kam ein Junge, ungefähr zehn Jahre alt, angelaufen und rief nach dem Hund. "Entschuldigung, das ist mein Hund Max," sagte er schüchtern. Anna lächelte und sagte: "Kein Problem, er ist sehr freundlich."

Der Junge setzte sich zu Anna und sie unterhielten sich über den Hund und das schöne Wetter. Er erzählte ihr, dass er jeden Samstag mit Max in den Park kommt, um zu spielen. Anna fand es schön, neue Leute kennenzulernen und sich zu unterhalten.

Nach einiger Zeit verabschiedete sich der Junge und Anna ging weiter spazieren. Sie genoss die frische Luft und die friedliche Atmosphäre. Schließlich fand sie eine Bank neben einem kleinen Teich und setzte sich, um die Enten zu beobachten.

Als der Nachmittag zu Ende ging, packte Anna ihre Sachen zusammen und machte sich auf den Weg nach Hause. Es war ein einfacher, aber sehr schöner Tag gewesen. Sie fühlte sich glücklich und entspannt. Manchmal sind es die kleinen Dinge im Leben, die den größten Unterschied machen.

Ich hoffe, diese Geschichte gefällt dir!

"""

# Step by Step

1. Receive the text
2. Split the text into sentences
3. POS tag each one of the tokens from each sentence and enrich it into the dictionary
   
   ```json
   {
      "NOUN": {"Tag": {"singular": "der Tag", "count": 2}, "Rucksack": {"singular": "der Rucksack", "count": 1}, ... },
      "VERB": {"beschloss": {"infinitive": "beschließen", "pras": "beschließt", "prat": "beschloss", "perf": "hat beschlossen"}}
   }
   ```

## Multilingual Universal Part-of-Speech Tagging in Flair (default model)

This is the default multilingual universal part-of-speech tagging model that ships with [Flair](https://github.com/flairNLP/flair/).

F1-Score: **96.87** (12 UD Treebanks covering English, German, French, Italian, Dutch, Polish, Spanish, Swedish, Danish, Norwegian, Finnish and Czech)

Predicts universal POS tags:

| **tag**                        | **meaning** |
|---------------------------------|-----------|
|ADJ |  adjective |
 |   ADP |  adposition |
 |   ADV |  adverb |
 |   AUX |  auxiliary |
 |   CCONJ |  coordinating conjunction |
 |   DET |  determiner |
 |   INTJ |  interjection |
 |   NOUN |  noun |
 |   NUM |  numeral |
 |   PART |  particle |
 |   PRON |  pronoun |
 |   PROPN |  proper noun |
 |   PUNCT |  punctuation |
 |   SCONJ |  subordinating conjunction |
 |   SYM |  symbol |
 |   VERB |  verb |
 |   X |  other |


In [132]:
# https://github.com/gambolputty/german-nouns
from german_nouns.lookup import Nouns

def singular(noun : str):
    """A function that outputs the singular of a noun

    Args:
        noun (_type_): _description_

    Returns:
        _type_: _description_
    """

    Worterbuch = Nouns()
    out = Worterbuch[noun]

    article = {
        'm': 'der',
        'f': 'die',
        'n': 'das'
    }

    try:
        out = out[0]
        out = article.get(out.get('genus')) + ' ' + out.get('flexion').get('nominativ singular')
    except:
        out = 'none'

    return out

tag_dict={
  "ADJ": {},
  "ADP": {},
  "ADV": {},
  "AUX": {},
  "CCONJ": {},
  "DET": {},
  "INTJ": {},
  "NOUN": {},
  "NUM": {},
  "PART": {},
  "PRON": {},
  "PROPN": {},
  "PUNCT": {},
  "SCONJ": {},
  "SYM": {},
  "VERB": {},
  "X": {},
}

# initialize sentence splitter
splitter = SegtokSentenceSplitter()
# use splitter to split text into list of sentences
sentences = splitter.split(text)
# print sentence
pprint(sentences)

# load tagger
pos_tagger = SequenceTagger.load("flair/upos-multi")
# predict POS tags
pos_tagger.predict(sentences)

THRESHOLD = 0.95
for sentence in sentences:
    for label in sentence.get_labels():
        
        token = label.data_point.text
        score = label.data_point.score
        tag = label.value
        
        token_dict = tag_dict[tag].get(token)
        
        if label.value == "NOUN" and score >= THRESHOLD:
            
            if tag_dict[tag].get(token) is None:
                #assignment on initial dict
                tag_dict[tag][token] = {"singular": singular(token), "count": 1}

            else:
                tag_dict[tag][token]["count"] = tag_dict[tag][token]["count"] + 1

        else:
            pass
    
tag_dict

2024-07-28 14:47:13,757 Warning: An empty Sentence was created! Are there empty strings in your dataset?


[Sentence[5]: "Ein besonderer Tag im Park",
 Sentence[17]: "Es war ein sonniger Samstagmorgen, als Anna beschloss, einen Tag im Park zu verbringen.",
 Sentence[15]: "Sie packte ihren Rucksack mit einer Decke, einem Buch und etwas zu essen.",
 Sentence[14]: "Als sie ankam, fand sie einen schönen Platz unter einem großen Baum.",
 Sentence[11]: "Anna setzte sich und begann, ihr Buch zu lesen.",
 Sentence[14]: "Die Vögel zwitscherten fröhlich, und die Sonne schien warm auf ihr Gesicht.",
 Sentence[18]: "Nach einer Weile hörte sie Kinder lachen und sah eine Gruppe von Kindern, die Fußball spielten.",
 Sentence[8]: "Es erinnerte sie an ihre eigene Kindheit.",
 Sentence[17]: "Plötzlich kam ein kleiner Hund auf sie zugelaufen und begann, an ihrer Decke zu schnüffeln.",
 Sentence[7]: "Anna lachte und streichelte den Hund.",
 Sentence[18]: "Bald darauf kam ein Junge, ungefähr zehn Jahre alt, angelaufen und rief nach dem Hund.",
 Sentence[13]: ""Entschuldigung, das ist mein Hund Max," sagte er sc

{'ADJ': {},
 'ADP': {},
 'ADV': {},
 'AUX': {},
 'CCONJ': {},
 'DET': {},
 'INTJ': {},
 'NOUN': {'Tag': {'singular': 'der Tag', 'count': 3},
  'Samstagmorgen': {'singular': 'der Samstagmorgen', 'count': 1},
  'Park': {'singular': 'der Park', 'count': 1},
  'Rucksack': {'singular': 'der Rucksack', 'count': 1},
  'Decke': {'singular': 'die Decke', 'count': 2},
  'Buch': {'singular': 'das Buch', 'count': 2},
  'Platz': {'singular': 'der Platz', 'count': 1},
  'Baum': {'singular': 'der Baum', 'count': 1},
  'Vögel': {'singular': 'der Vogel', 'count': 1},
  'Sonne': {'singular': 'die Sonne', 'count': 1},
  'Gesicht': {'singular': 'das Gesicht', 'count': 1},
  'Weile': {'singular': 'die Weile', 'count': 1},
  'Kinder': {'singular': 'das Kind', 'count': 1},
  'Gruppe': {'singular': 'die Gruppe', 'count': 1},
  'Kindern': {'singular': 'das Kind', 'count': 1},
  'Kindheit': {'singular': 'die Kindheit', 'count': 1},
  'Hund': {'singular': 'der Hund', 'count': 5},
  'Junge': {'singular': 'der Jun

# Solution 2

## Improvements:
1. Consider trennbare verben
   1. A initial Idea would be to use the PTKVZ(Separable prefix) Tag and join it with the Verb (https://www.sketchengine.eu/german-stts-part-of-speech-tagset/)
2. Take the full sentence which the token is inserted to create examples for the normalized tokens.


In [27]:
doc = nlp(text)
token_span = doc[2:3]
token_span

stehe

In [28]:
token_span.sent

Ich stehe jeden Morgen um sechs Uhr auf.

In [1]:
import spacy

import numpy as np
import pandas as pd
pd.set_option('display.max_colwidth', None)
pd.set_option('display.expand_frame_repr', False)

from googletrans import Translator

nlp = spacy.load('de_core_news_md')

text = """
Ein besonderer Tag im Park

Es war ein sonniger Samstagmorgen, als Anna beschloss, einen Tag im Park zu verbringen. Sie packte ihren Rucksack mit einer Decke, einem Buch und etwas zu essen. Als sie ankam, fand sie einen schönen Platz unter einem großen Baum.

Anna setzte sich und begann, ihr Buch zu lesen. Die Vögel zwitscherten fröhlich, und die Sonne schien warm auf ihr Gesicht. Nach einer Weile hörte sie Kinder lachen und sah eine Gruppe von Kindern, die Fußball spielten. Es erinnerte sie an ihre eigene Kindheit.

Plötzlich kam ein kleiner Hund auf sie zugelaufen und begann, an ihrer Decke zu schnüffeln. Anna lachte und streichelte den Hund. Bald darauf kam ein Junge, ungefähr zehn Jahre alt, angelaufen und rief nach dem Hund. "Entschuldigung, das ist mein Hund Max," sagte er schüchtern. Anna lächelte und sagte: "Kein Problem, er ist sehr freundlich."

Der Junge setzte sich zu Anna und sie unterhielten sich über den Hund und das schöne Wetter. Er erzählte ihr, dass er jeden Samstag mit Max in den Park kommt, um zu spielen. Anna fand es schön, neue Leute kennenzulernen und sich zu unterhalten.

Nach einiger Zeit verabschiedete sich der Junge und Anna ging weiter spazieren. Sie genoss die frische Luft und die friedliche Atmosphäre. Schließlich fand sie eine Bank neben einem kleinen Teich und setzte sich, um die Enten zu beobachten.

Als der Nachmittag zu Ende ging, packte Anna ihre Sachen zusammen und machte sich auf den Weg nach Hause. Es war ein einfacher, aber sehr schöner Tag gewesen. Sie fühlte sich glücklich und entspannt. Manchmal sind es die kleinen Dinge im Leben, die den größten Unterschied machen.

Ich hoffe, diese Geschichte gefällt dir!

"""

# text = """
# Ich stehe jeden Morgen um sechs Uhr auf.
# Wir kaufen am Samstag im Supermarkt ein.
# Kannst du mich heute Abend anrufen?
# Bitte mach das Fenster auf.
# Kommst du mit ins Kino?
# """

text = """In Deutschland gibt es 35 Parteien, die bei der Europawahl antreten. Manche davon kennen wir aus der deutschen Regierung, natürlich treten SPD, FDP und Die Grünen an."""

clean_text = text.replace('\n','')
doc = nlp(clean_text)
df_doc = pd.DataFrame([(_, token, token.sent, token.lemma_, token.pos_, token.tag_, token.morph) for _, token in enumerate(doc)],
                      columns = ['position', 'token', 'sentence', 'lemma', 'pos', 'tag', 'morph'])

morph_df = pd.DataFrame([m.to_dict() for m in df_doc['morph']])
morph_df = morph_df[np.sort(morph_df.columns)]

df_doc.drop(columns=['morph'], inplace=True)
out_df = pd.concat([df_doc, morph_df], axis=1)

#out_df['Definite'] = out_df['Definite'].replace({'Def': 1, 'Ind': 0})

gender_article = {
    'Fem':'die ',
    'Neut':'das ',
    'Masc':'der ',
}

#this is the noun in nominative singular case
try:
    out_df['norm_noun'] = np.where(out_df['pos'] == 'NOUN',
                                                out_df['Gender'].replace(gender_article) + out_df['lemma'].astype(str),
                                                np.nan
                                                )
except:
    print('No NOUN detected in text')

informative_pos = ["ADJ", "VERB", "ADV"]
for pos in informative_pos:
    out_df[f'norm_{pos.lower()}'] = np.where(out_df['pos'] == pos,
                                            out_df['lemma'],
                                            np.nan
                                            )

out_df.rename(columns = {'Definite': 'is_definite',
                         'Gender': 'gender'}, inplace=True)
out_df.columns = [col.lower() for col in out_df.columns]
out_df = out_df.fillna('').astype(str)

norm_cols = [col for col in out_df.columns if col[:5] == 'norm_']
out_df['norm_token'] = out_df[norm_cols].sum(axis=1)

out_df

,position,token,sentence,lemma,pos,tag,case,is_definite,degree,gender,...,number,person,prontype,tense,verbform,norm_noun,norm_adj,norm_verb,norm_adv,norm_token
0,0,In,"In Deutschland gibt es 35 Parteien, die bei der Europawahl antreten.",in,ADP,APPR,,,,,...,,,,,,,,,,
1,1,Deutschland,"In Deutschland gibt es 35 Parteien, die bei der Europawahl antreten.",Deutschland,PROPN,NE,Dat,,,Neut,...,Sing,,,,,,,,,
2,2,gibt,"In Deutschland gibt es 35 Parteien, die bei der Europawahl antreten.",geben,VERB,VVFIN,,,,,...,Sing,3,,Pres,Fin,,,geben,,geben
3,3,es,"In Deutschland gibt es 35 Parteien, die bei der Europawahl antreten.",es,PRON,PPER,Nom,,,Neut,...,Sing,3,Prs,,,,,,,
4,4,35,"In Deutschland gibt es 35 Parteien, die bei der Europawahl antreten.",35,NUM,CARD,,,,,...,,,,,,,,,,
5,5,Parteien,"In Deutschland gibt es 35 Parteien, die bei der Europawahl antreten.",Partei,NOUN,NN,Acc,,,Fem,...,Plur,,,,,die Partei,,,,die Partei
6,6,",","In Deutschland gibt es 35 Parteien, die bei der Europawahl antreten.",--,PUNCT,"$,",,,,,...,,,,,,,,,,
7,7,die,"In Deutschland gibt es 35 Parteien, die bei der Europawahl antreten.",der,PRON,PRELS,Nom,,,Fem,...,Plur,,Rel,,,,,,,
8,8,bei,"In Deutschland gibt es 35 Parteien, die bei der Europawahl antreten.",bei,ADP,APPR,,,,,...,,,,,,,,,,
9,9,der,"In Deutschland gibt es 35 Parteien, die bei der Europawahl antreten.",der,DET,ART,Dat,Def,,Fem,...,Sing,,Art,,,,,,,


In [16]:
tb_pos1 = out_df[out_df['tag'] ==  'PTKVZ']['position'].to_numpy()
tb_pos2 = out_df[(out_df['pos'] ==  'VERB') & (out_df['tag'] ==  'VVFIN')]['position'].to_numpy()


cross_join = [(x, y) for x in tb_pos1 for y in tb_pos2 if x > y]
cross_join

[('30', '2'), ('30', '15'), ('30', '23')]

In [12]:
np.cross(tb_pos1, tb_pos2) 

ValueError: incompatible dimensions for cross product
(dimension must be 2 or 3)

In [13]:



print(cross_join)



[-3  6 -3]


In [87]:
only_important = out_df['norm_token'] != ""
token_df = out_df[only_important]
token_df

,position,token,sentence,lemma,pos,tag,case,is_definite,degree,gender,...,poss,prontype,reflex,tense,verbform,norm_noun,norm_adj,norm_verb,norm_adv,norm_token
1,1,besonderer,"Ein besonderer Tag im ParkEs war ein sonniger Samstagmorgen, als Anna beschloss, einen Tag im Park zu verbringen.",besonderer,ADJ,ADJA,Nom,,Pos,Masc,...,,,,,,,besonderer,,,besonderer
2,2,Tag,"Ein besonderer Tag im ParkEs war ein sonniger Samstagmorgen, als Anna beschloss, einen Tag im Park zu verbringen.",Tag,NOUN,NN,Nom,,,Masc,...,,,,,,der Tag,,,,der Tag
7,7,sonniger,"Ein besonderer Tag im ParkEs war ein sonniger Samstagmorgen, als Anna beschloss, einen Tag im Park zu verbringen.",sonnig,ADJ,ADJA,Nom,,Pos,Masc,...,,,,,,,sonnig,,,sonnig
8,8,Samstagmorgen,"Ein besonderer Tag im ParkEs war ein sonniger Samstagmorgen, als Anna beschloss, einen Tag im Park zu verbringen.",Samstagmorgen,NOUN,NN,Nom,,,Masc,...,,,,,,der Samstagmorgen,,,,der Samstagmorgen
15,15,Tag,"Ein besonderer Tag im ParkEs war ein sonniger Samstagmorgen, als Anna beschloss, einen Tag im Park zu verbringen.",Tag,NOUN,NN,Acc,,,Masc,...,,,,,,der Tag,,,,der Tag
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
312,312,Unterschied,"Manchmal sind es die kleinen Dinge im Leben, die den größten Unterschied machen.",Unterschied,NOUN,NN,Acc,,,Masc,...,,,,,,der Unterschied,,,,der Unterschied
313,313,machen,"Manchmal sind es die kleinen Dinge im Leben, die den größten Unterschied machen.",machen,VERB,VVINF,,,,,...,,,,,Inf,,,machen,,machen
316,316,hoffe,"Ich hoffe, diese Geschichte gefällt dir!",hoffen,VERB,VVFIN,,,,,...,,,,Pres,Fin,,,hoffen,,hoffen
319,319,Geschichte,"Ich hoffe, diese Geschichte gefällt dir!",Geschichte,NOUN,NN,Nom,,,Fem,...,,,,,,die Geschichte,,,,die Geschichte


In [88]:
#group by token and normalized token and bring all sentences
token_df = token_df.groupby(['token', 'norm_token'], as_index=False)['sentence'].apply(list)
token_df

,token,norm_token,sentence
0,Atmosphäre,die Atmosphäre,[Sie genoss die frische Luft und die friedliche Atmosphäre.]
1,Bald,bald,"[Bald darauf kam ein Junge, ungefähr zehn Jahre alt, angelaufen und rief nach dem Hund.]"
2,Bank,die Bank,"[Schließlich fand sie eine Bank neben einem kleinen Teich und setzte sich, um die Enten zu beobachten.]"
3,Baum,der Baum,"[Als sie ankam, fand sie einen schönen Platz unter einem großen Baum.]"
4,Buch,das Buch,"[Sie packte ihren Rucksack mit einer Decke, einem Buch und etwas zu essen., Anna setzte sich und begann, ihr Buch zu lesen.]"
...,...,...,...
104,warm,warm,"[Die Vögel zwitscherten fröhlich, und die Sonne schien warm auf ihr Gesicht.]"
105,weiter,weiter,[Nach einiger Zeit verabschiedete sich der Junge und Anna ging weiter spazieren.]
106,zugelaufen,zugelaufen,"[Plötzlich kam ein kleiner Hund auf sie zugelaufen und begann, an ihrer Decke zu schnüffeln.]"
107,zusammen,zusammen,"[Als der Nachmittag zu Ende ging, packte Anna ihre Sachen zusammen und machte sich auf den Weg nach Hause.]"


In [89]:
from googletrans import Translator
from tqdm import tqdm

#card_df will be based on the normalized_tokens
card_df = token_df.groupby(['norm_token'],as_index=False)['sentence'].sum()

translator = Translator()
dest_language = 'en'
card_dest = []

tokens_to_translate = card_df['norm_token'].values
sentences_to_translate = card_df['sentence'].values

for card_input in tqdm(tokens_to_translate):
    resp = translator.translate(card_input, dest=dest_language)
    card_dest.append(resp.text)

# sent_dest = []
# for sentences in sentences_to_translate:
    
#     for sent in sentences:

table_csv = pd.DataFrame(
    zip(tokens_to_translate, card_dest, sentences_to_translate), 
    columns = ['card_de', f'card_{dest_language}', 'sentences_de'])

100%|██████████| 99/99 [02:11<00:00,  1.32s/it]


,card_de,card_en,sentences_de
0,alt,alt,"[Bald darauf kam ein Junge, ungefähr zehn Jahre alt, angelaufen und rief nach dem Hund.]"
1,ankommen,arrive,"[Als sie ankam, fand sie einen schönen Platz unter einem großen Baum.]"
2,anlaufen,start,"[Bald darauf kam ein Junge, ungefähr zehn Jahre alt, angelaufen und rief nach dem Hund.]"
3,bald,bald,"[Bald darauf kam ein Junge, ungefähr zehn Jahre alt, angelaufen und rief nach dem Hund.]"
4,beginnen,to start,"[Anna setzte sich und begann, ihr Buch zu lesen., Plötzlich kam ein kleiner Hund auf sie zugelaufen und begann, an ihrer Decke zu schnüffeln.]"
...,...,...,...
94,warm,warm,"[Die Vögel zwitscherten fröhlich, und die Sonne schien warm auf ihr Gesicht.]"
95,weiter,further,[Nach einiger Zeit verabschiedete sich der Junge und Anna ging weiter spazieren.]
96,zugelaufen,overflowed,"[Plötzlich kam ein kleiner Hund auf sie zugelaufen und begann, an ihrer Decke zu schnüffeln.]"
97,zusammen,together,"[Als der Nachmittag zu Ende ging, packte Anna ihre Sachen zusammen und machte sich auf den Weg nach Hause.]"


In [93]:
table_csv.sample(15)

,card_de,card_en,sentences_de
30,der Teich,the pond,"[Schließlich fand sie eine Bank neben einem kleinen Teich und setzte sich, um die Enten zu beobachten.]"
8,das Buch,the book,"[Sie packte ihren Rucksack mit einer Decke, einem Buch und etwas zu essen., Anna setzte sich und begann, ihr Buch zu lesen.]"
94,warm,warm,"[Die Vögel zwitscherten fröhlich, und die Sonne schien warm auf ihr Gesicht.]"
38,die Geschichte,the history,"[Ich hoffe, diese Geschichte gefällt dir!]"
22,der Junge,the young,"[Bald darauf kam ein Junge, ungefähr zehn Jahre alt, angelaufen und rief nach dem Hund., Anna lächelte und sagte: ""Kein Problem, er ist sehr freundlich.""Der Junge setzte sich zu Anna und sie unterhielten sich über den Hund und das schöne Wetter., Nach einiger Zeit verabschiedete sich der Junge und Anna ging weiter spazieren.]"
7,darauf,on it,"[Bald darauf kam ein Junge, ungefähr zehn Jahre alt, angelaufen und rief nach dem Hund.]"
88,spielen,spielen,"[Er erzählte ihr, dass er jeden Samstag mit Max in den Park kommt, um zu spielen., Nach einer Weile hörte sie Kinder lachen und sah eine Gruppe von Kindern, die Fußball spielten.]"
52,finden,find,"[Als sie ankam, fand sie einen schönen Platz unter einem großen Baum., Anna fand es schön, neue Leute kennenzulernen und sich zu unterhalten., Schließlich fand sie eine Bank neben einem kleinen Teich und setzte sich, um die Enten zu beobachten.]"
50,erzählen,tell,"[Er erzählte ihr, dass er jeden Samstag mit Max in den Park kommt, um zu spielen.]"
47,einfach,simply,"[Es war ein einfacher, aber sehr schöner Tag gewesen.]"
